# Import required libraries and modules

In [29]:
from pathlib import Path
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import keras
import cv2
import tensorflow as tf
import keras
from keras.layers import Conv2D, Dense, Flatten, MaxPool2D
from keras.models import Sequential, load_model
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.preprocessing import LabelEncoder
from keras.layers import Lambda
from PIL import Image
import cv2
from tensorflow.keras import layers
from keras.utils import to_categorical

# Create a custom generator

This makes sure, that the data is not loaded at once into the memory, when training the model

In [30]:
class CustomGenerator(keras.utils.Sequence):
    def __init__(self, image_filenames, labels, batch_size):
        self.image_filenames = image_filenames
        self.labels = labels
        self.batch_size = batch_size

    def __len__(self):
        return (np.ceil(len(self.image_filenames) / float(self.batch_size))).astype(int)

    def __getitem__(self, idx):
        batch_x = self.image_filenames[idx * self.batch_size: (idx + 1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size: (idx + 1) * self.batch_size]        
        return np.array([
            (cv2.imread(str(file_name)))
            for file_name in batch_x]), np.array(batch_y)

# Load the data

In [31]:
path = Path("test_data")
df = pd.read_csv(path/'artists.csv')

# Check the distribution of the data

In [32]:
artists_df = df[['name', 'paintings']].groupby(['name'], as_index = False).sum()
names = artists_df.sort_values('paintings', ascending = False)[:50]
print(names)

                         name  paintings
48           Vincent van Gogh        877
10                Edgar Degas        702
33              Pablo Picasso        439
38      Pierre-Auguste Renoir        336
0              Albrecht Dürer        328
35               Paul Gauguin        311
15             Francisco Goya        291
42                  Rembrandt        262
1               Alfred Sisley        259
46                     Titian        255
30               Marc Chagall        239
43              Rene Magritte        194
2           Amedeo Modigliani        193
36                  Paul Klee        188
21              Henri Matisse        186
4                 Andy Warhol        181
32             Mikhail Vrubel        171
45          Sandro Botticelli        164
29          Leonardo da Vinci        143
37          Peter Paul Rubens        141
44              Salvador Dali        139
24           Hieronymus Bosch        137
40             Pieter Bruegel        134
9             Di

# Create a list with all paintings and artists

In [33]:
images_dir = Path(path/'images/images')

painting_list = []
for path, subdirs, files in os.walk(images_dir):
    for name in files:
        img = os.path.join(path, name)
        painting_list.extend([img])

#only works on windows
artist_list = []
for painting in painting_list:
    artist = painting.split('\\')[3]
    artist_list.extend([artist])

# Create X and y

In [34]:
y = np.array(artist_list)  

encoder = LabelEncoder()
y = encoder.fit_transform(pd.Series(y))
classes = list(encoder.classes_)
for i in range(len(classes)):
    print(i, ' represents ', classes[i])

y = keras.utils.to_categorical(y, len(classes))
print(y)

X = np.array(painting_list) 

0  represents  Rembrandt
1  represents  Rene_Magritte
2  represents  Salvador_Dali
3  represents  Sandro_Botticelli
4  represents  Titian
5  represents  Vasiliy_Kandinskiy
6  represents  Vincent_van_Gogh
7  represents  William_Turner
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


# Split data into train, validation and test data

In [35]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=1)

# Prepare generators for training

In [36]:
batch_size = 16
training_batch_generator = CustomGenerator(X_train, y_train, batch_size)
validation_batch_generator = CustomGenerator(X_val, y_val, batch_size)
training_size = len(X_train)
validation_size = len(X_val)
test_size = len(X_test)
input_shape = (cv2.imread(str(painting_list[0]))).shape
print(input_shape)

(224, 224, 3)


# A model based on VGG-16's architecture

In [43]:
def make_vgg16_model(input_shape, num_classes):
    # Generate the model
    model = Sequential()

    # Layer 1: Convolutional
    model.add(Conv2D(input_shape=input_shape, filters=64, kernel_size=(3, 3),
                     padding='same', activation='relu'))

    # Layer 2: Convolutional
    model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'))

    # Layer 3: MaxPooling
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))

    # Layer 4: Convolutional
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu'))
    # Layer 5: Convolutional
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu'))
    # Layer 6: MaxPooling
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))

    # Layer 7: Convolutional
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu'))
    # Layer 8: Convolutional
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu'))
    # Layer 9: Convolutional
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu'))
    # Layer 10: MaxPooling
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))

    # Layer 11: Convolutional
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu'))
    # Layer 12: Convolutional
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu'))
    # Layer 13: Convolutional
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu'))
    # Layer 14: MaxPooling
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))

    # Layer 15: Convolutional
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu'))
    # Layer 16: Convolutional
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu'))
    # Layer 17: Convolutional
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu'))
    # Layer 18: MaxPooling
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))

    # Layer 19: Flatten
    model.add(Flatten())
    # Layer 20: Fully Connected Layer
    model.add(Dense(units=4096, activation='relu'))
    # Layer 21: Fully Connected Layer
    model.add(Dense(units=4096, activation='relu'))
    # Layer 22: Softmax Layer
    model.add(Dense(units=num_classes, activation='softmax'))
    print(model.summary())
    return model

# A model based on an image classification tutorial by keras

In [38]:
def make_keras_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)

    # Entry block
    x = layers.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(128, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    
    activation = "softmax"

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation=activation)(x)
    return keras.Model(inputs, outputs)

# Train keras-like model

In [39]:
model_keras = make_keras_model(input_shape, len(classes))
print(model_keras.summary())

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_2 (Rescaling)        (None, 224, 224, 3)  0           ['input_3[0][0]']                
                                                                                                  
 conv2d_8 (Conv2D)              (None, 112, 112, 12  3584        ['rescaling_2[0][0]']            
                                8)                                                                
                                                                                            

In [40]:
model_keras.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
model_keras.fit_generator(generator=training_batch_generator,
                    steps_per_epoch=int(training_size // batch_size),
                    epochs=6,
                    verbose=1,
                    validation_data=validation_batch_generator,
                    validation_steps=int(validation_size // batch_size)
                   )

Epoch 1/6


C:\Users\justu\AppData\Local\Temp\ipykernel_20648\1483329134.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_keras.fit_generator(generator=training_batch_generator,


102/102 [==============================] - 269s 3s/step - loss: 1.5084 - accuracy: 0.4981 - val_loss: 1.9930 - val_accuracy: 0.0885
Epoch 2/6
102/102 [==============================] - 259s 3s/step - loss: 1.2868 - accuracy: 0.5543 - val_loss: 2.0793 - val_accuracy: 0.1719
Epoch 3/6
102/102 [==============================] - 259s 3s/step - loss: 1.1529 - accuracy: 0.6049 - val_loss: 2.3723 - val_accuracy: 0.1719
Epoch 4/6
102/102 [==============================] - 256s 3s/step - loss: 1.0761 - accuracy: 0.6210 - val_loss: 2.4821 - val_accuracy: 0.2031
Epoch 5/6
102/102 [==============================] - 265s 3s/step - loss: 1.0367 - accuracy: 0.6327 - val_loss: 1.6959 - val_accuracy: 0.3438
Epoch 6/6
102/102 [==============================] - 269s 3s/step - loss: 0.9405 - accuracy: 0.6698 - val_loss: 1.2292 - val_accuracy: 0.5729


In [41]:
model_keras.save(Path('model_keras_test.h5'))

# Train VGG16-like model

In [44]:
model_vgg16 = make_vgg16_model(input_shape, len(classes))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_25 (Conv2D)          (None, 224, 224, 64)      1792      
                                                                 
 conv2d_26 (Conv2D)          (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 112, 112, 64)     0         
 g2D)                                                            
                                                                 
 conv2d_27 (Conv2D)          (None, 112, 112, 128)     73856     
                                                                 
 conv2d_28 (Conv2D)          (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 56, 56, 128)      0         
 g2D)                                                 

In [45]:
model_vgg16.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
model_vgg16.fit_generator(generator=training_batch_generator,
                    steps_per_epoch=int(training_size // batch_size),
                    epochs=6,
                    verbose=1,
                    validation_data=validation_batch_generator,
                    validation_steps=int(validation_size // batch_size)
                   )

Epoch 1/6


C:\Users\justu\AppData\Local\Temp\ipykernel_20648\737468964.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_vgg16.fit_generator(generator=training_batch_generator,


102/102 [==============================] - 504s 5s/step - loss: 48.5360 - accuracy: 0.3957 - val_loss: 1.7226 - val_accuracy: 0.4531
Epoch 2/6
102/102 [==============================] - 531s 5s/step - loss: 1.7534 - accuracy: 0.4333 - val_loss: 1.7199 - val_accuracy: 0.4531
Epoch 3/6
102/102 [==============================] - 499s 5s/step - loss: 1.7396 - accuracy: 0.4346 - val_loss: 1.7169 - val_accuracy: 0.4531
Epoch 4/6
102/102 [==============================] - 516s 5s/step - loss: 1.7367 - accuracy: 0.4352 - val_loss: 1.7286 - val_accuracy: 0.4531
Epoch 5/6
102/102 [==============================] - 520s 5s/step - loss: 1.7403 - accuracy: 0.4340 - val_loss: 1.7214 - val_accuracy: 0.4531
Epoch 6/6
102/102 [==============================] - 524s 5s/step - loss: 1.7401 - accuracy: 0.4340 - val_loss: 1.7254 - val_accuracy: 0.4531


In [ ]:
model_vgg16.save(Path('model_vgg16_test.h5'))